In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
sentence_length = 106

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS", "True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS", "True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

In [ ]:
from tooling.config import Experiment, Transformation

spellchecked_experiment_config = Experiment(
    name="SPELLCHECKED_Config",
    iterations=5,
    force=False,
    dataset="spellchecked",
    lower_case=True,
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Domain_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

In [ ]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)

Path("./src/service/models/").mkdir(parents=True, exist_ok=True)

hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

## Train BERT First Stage Model with SpellChecked Datasets


In [ ]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(spellchecked_experiment_config)
bert_1_experiment_config.name = "SPELLCHECKED_LEVELS"
bert_1_experiment_config.force = False

bert_1_config = BERT(
    max_len=123,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.0005,
    validation_batch_size=64,
)


bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit=pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")

## Train BERT E2E Model with SpellChecked Datasets


In [ ]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(spellchecked_experiment_config)
bert_experiment_config.name = "SPELLCHECKED_CATEGORIES"

bert_config = BERT(
    learning_rate_bert=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit=pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")